# MindSpore实战一，MNIST手写体识别

数据集介绍：
* MNIST数据集来自美国国家标准与技术研究所，National Institute of Standards and Technology(NIST),数据集由来自250个不同人手写的数字构成，其中50%是高中学生，50%来自人口普查局（the Census Bureau）的工作人员。

* 训练集：60000，测试集：10000

* MNIST数据集可在 http://yann.lecun.com/exdb/mnist/ 获取。

<img src="image/01.png">

本实验使用MindSpore深度学习框架，进行网络搭建、数据处理、网络训练和测试，完成MNIST手写体识别任务。

实验流程：

<img src="image/02.png">

## 环境准备
* MindSpore模块主要用于本次实验卷积神经网络的构建，包括很多子模块。
    * mindspore.dataset：包括MNIST数据集的载入与处理，也可以自定义数据集。

    * mindspore.common：包中会有诸如type形态转变、权重初始化等的常规工具。

    * mindspore.nn：主要包括网络可能涉及到的各类网络层，诸如卷积层、池化层、全连接层，也包括损失函数，激活函数等。
    
    * Model：承载网络结构，并能够调用优化器、损失函数、评价指标。


本实验需要以下第三方库：
1. MindSpore 1.7
2. Numpy 1.17.5

In [1]:
# mindspore.dataset
import mindspore.dataset as ds # 数据集的载入
import mindspore.dataset.transforms.c_transforms as C # 常用转化算子
import mindspore.dataset.vision.c_transforms as CV # 图像转化算子

# mindspore.common
from mindspore.common import dtype as mstype # 数据形态转换
from mindspore.common.initializer import Normal # 参数初始化

# mindspore.nn
import mindspore.nn as nn # 各类网络层都在nn里面
from mindspore.nn.metrics import Accuracy # 测试模型用


from mindspore import Model # 承载网络结构


# os模块处理数据路径用
import os

# numpy
import numpy as np

## 数据处理

定义数据预处理函数。

函数功能包括：
1. 加载数据集
1. 打乱数据集
1. 图像特征处理（标准化、通道转换等）
3. 批量输出数据
4. 重复

In [2]:
def create_dataset(data_path, batch_size=32):
    """ 
    数据预处理与批量输出的函数
    
    Args:
        data_path: 数据路径
        batch_size: 批量大小
    """
    
    # 定义数据集
    data = ds.MnistDataset(data_path)
    
    # 打乱数据集
    data = data.shuffle(buffer_size=10000)
    
    # 数据标准化参数
    # MNIST数据集的 mean = 33.3285，std = 78.5655
    mean, std = 33.3285, 78.5655 

    # 定义算子
    nml_op = lambda x : np.float32((x-mean)/std) # 数据标准化，image = (image-mean)/std
    hwc2chw_op = CV.HWC2CHW() # 通道前移（为配适网络，CHW的格式可最佳发挥昇腾芯片算力）
    type_cast_op = C.TypeCast(mstype.int32) # 原始数据的标签是unint，计算损失需要int

    # 算子运算
    data = data.map(operations=type_cast_op, input_columns='label')
    data = data.map(operations=nml_op, input_columns='image')
    data = data.map(operations=hwc2chw_op, input_columns='image')

    # 批处理
    data = data.batch(batch_size)
    
    # 重复
    data = data.repeat(1)

    return data

## 网络定义
### 参考LeNet网络结构，构建网络
LeNet-5出自论文《Gradient-Based Learning Applied to Document Recognition》，原本是一种用于手写体字符识别的非常高效的卷积神经网络，包含了深度学习的基本模块：卷积层，池化层，全连接层。

本实验将参考LeNet论文，建立以下网络：
<img src="image/03.png">

1.	INPUT（输入层） ：输入28∗28的图片。
2.	C1（卷积层）：选取6个5∗5卷积核(不包含偏置)，得到6个特征图，每个特征图的一个边为28−5+1=24。
3.	S2（池化层）：池化层是一个下采样层，输出12∗12∗6的特征图。
4.	C3（卷积层）：选取16个大小为5∗5卷积核，得到特征图大小为8∗8∗16。
5.	S4（池化层）：窗口大小为2∗2，输出4∗4∗16的特征图。
6.	F5（全连接层）：120个神经元。
7.	F6（全连接层）：84个神经元。
8.	OUTPUT（输出层）：10个神经元，10分类问题。


In [3]:
class LeNet5(nn.Cell):
    
    # 定义算子
    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        # 卷积层
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        
        # 全连接层
        self.fc1 = nn.Dense(4 * 4 * 16, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        
        # 激活函数
        self.relu = nn.ReLU()
        
        # 最大池化成
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 网络展开
        self.flatten = nn.Flatten()
        
    # 建构网络
    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

## 模型训练

载入数据集

In [4]:
train_path = os.path.join('data','train') # 训练集路径
train_data = create_dataset(train_path) # 定义训练数据集

test_path = os.path.join('data','test') # 测试集路径
test_data = create_dataset(test_path) # 定义测试数据集

定义网络、损失函数、优化器、模型

In [5]:
# 网络
net = LeNet5()

# 损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

# 优化器
lr = 0.01
momentum = 0.9
net_opt = nn.Momentum(net.trainable_params(), lr, momentum)

# 模型
model = Model(net, net_loss, net_opt, metrics={'accuracy': Accuracy()})

训练模型

In [6]:
model.train(3, train_data) # 训练3个epoch

## 模型评估
查看模型在测试集的准确率

In [7]:
model.eval(test_data) # 测试网络

{'accuracy': 0.9877}

## 效果展示

In [8]:
data_path=os.path.join('data', 'test')

ds_test_demo = create_dataset(test_path, batch_size=1)

for i, dic in enumerate(ds_test_demo.create_dict_iterator()):
    input_img = dic['image']
    output = model.predict(input_img)
    predict = np.argmax(output.asnumpy(),axis=1)[0]
    if i>9:
        break
    print('True: %s, Predicted: %s'%(dic['label'], predict))


True: [9], Predicted: 9
True: [9], Predicted: 9
True: [8], Predicted: 0
True: [0], Predicted: 0
True: [6], Predicted: 6
True: [7], Predicted: 7
True: [9], Predicted: 9
True: [4], Predicted: 4
True: [7], Predicted: 7
True: [0], Predicted: 0


## 模型保存

In [9]:
from mindspore import save_checkpoint,load_checkpoint, load_param_into_net
save_checkpoint(net,"lenet5.ckpt")

## 思考
1. 请描述MindSpore的基础数据处理流程。
    * 答：数据加载 > shuffle > map > batch > repeat
2. 定义网络时需要继承哪一个基类？
    * 答：mindspore.nn.Cell
3. 定义网络时有哪些必须编写哪两个函数？
    * 答：\_\_init__()，construct()。
4. 思考3中提到的两个函数有什么用途？
    * 答：一般会在\_\_init__()中定义算子，然后在construct()中定义网络结构。\_\_init__()中的语句由Python解析执行；construct()中的语句由MindSpore接管，有语法限制；

